## Informações Gerais

Notebook produzido no contexto do projeto de pesquisa Análise de Dados Acadêmicos para estudo da Evasão Escolar, conduzido pelo Campus Viana em parceria com o Campus Monte Castelo, de 2020 a 2021. 

- Bolsista responsável: Arthur Mota França (MTC)
- Orientador: Prof. Josenildo Costa da Silva (MTC)
- Coordenador do Projeto: Prof. Francisco Silva (VIA)

# Instalações

In [ ]:
#!pip install -U pandas-profiling
!pip install category_encoders

     |████████████████████████████████| 80 kB 3.7 MB/s 


# Importações

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from category_encoders import *

from google.colab import drive
from pandas_profiling import ProfileReport
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from yellowbrick.classifier import ConfusionMatrix

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


# Funções

# Leitura dos Datasets

In [ ]:
# autenticar usuário
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"1hCvPIPWdcu3qXjmvEIbXZjqxVqSesbDl"})   # replace the id with id of file you want to access
downloaded.GetContentFile('matriculas.csv') 
dados_matr=pd.read_csv('matriculas.csv')


downloaded = drive.CreateFile({'id':"1psWRNGWGpz4K0Oeo1YcC833JI3TLcqKc"})   # replace the id with id of file you want to access
downloaded.GetContentFile('historico.csv') 
dados_hist=pd.read_csv('historico.csv')

# Pré-Processamento

Padronização dos nomes para minusculas

In [ ]:
#padronização dos nomes das tabelas para minusculas, para facilitar a manipulaçáo
dados_hist.columns=dados_hist.columns.str.lower()
dados_matr.columns=dados_matr.columns.str.lower()

Geração de relatorios dos nossos datasets usando Profile Report

In [ ]:
# profile = ProfileReport(dados_matr, title='Profiling Dados de Matricula',html={'style':{'full_width':True}})
# profile.to_notebook_iframe()

In [ ]:
# profile = ProfileReport(dados_hist, title='Profiling Dados Historicos',html={'style':{'full_width':True}})
# profile.to_notebook_iframe()

In [ ]:
dados_matr.describe()

,alunoid,anoingresso,periodoingresso,rendabruta,ira,idade,quantidade_computadores,quantidade_notebooks,qtd_filhos
count,34521.000000,34521.000000,34521.000000,3.452100e+04,34521.000000,34521.000000,31235.000000,31805.000000,34521.000000
mean,95245.318328,2017.882767,1.125576,3.792680e+05,5.555186,21.742418,0.160205,0.347650,0.238869
std,22833.352063,1.807386,0.335286,3.806605e+07,3.575950,44.287785,0.760202,0.554507,0.747113
min,5.000000,1998.000000,1.000000,-9.630000e+02,0.000000,-7979.000000,0.000000,0.000000,0.000000
25%,78940.000000,2017.000000,1.000000,2.000000e+02,1.270000,18.000000,0.000000,0.000000,0.000000
50%,97158.000000,2018.000000,1.000000,9.440000e+02,7.370000,20.000000,0.000000,0.000000,0.000000
75%,112922.000000,2019.000000,1.000000,1.498000e+03,8.190000,24.000000,0.000000,1.000000,0.000000
max,134472.000000,2021.000000,11.000000,4.960941e+09,79.000000,76.000000,100.000000,16.000000,13.000000


Manipulação de nosso dataset para tratar valores ausentes e inconsistentes

In [ ]:
#carga horaria negativa
dados_hist.loc[dados_hist.percentual_carga_horaria_frequentada < 0,"percentual_carga_horaria_frequentada"]=0

In [ ]:
#renda bruta negativa e muito alta, escolhemos limitar a 100k
dados_matr.loc[dados_matr.rendabruta < 0,"rendabruta"]=9.756800e+02
dados_matr.loc[dados_matr.rendabruta > 100000,"rendabruta"]=9.756800e+02

In [ ]:
#valores muito baixos
dados_matr.loc[dados_matr.idade < 14,"idade"]=18

In [ ]:
#muitos valores nulos, substituidos por zero
dados_matr.loc[dados_matr.quantidade_computadores.isnull(),"quantidade_computadores"]=0

In [ ]:
#muitos valores nulos, substituidos por zero
dados_matr.loc[dados_matr.quantidade_notebooks.isnull(),"quantidade_notebooks"]=0

In [ ]:
#tinhamos valores nulos para media final de varios alunos, para resolver esse problema, resolvemos usar alguns imputadores de valores. 
#Foram criados tres novas colunas, usando estrategias diferentes para completar o dataset
dados_hist["media_final_mean"]=dados_hist["media_final_disciplina"]
dados_hist["media_final_median"]=dados_hist["media_final_disciplina"]
dados_hist["media_final_frequency"]=dados_hist["media_final_disciplina"]
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
dados_hist["media_final_mean"] = imp.fit_transform(dados_hist[["media_final_mean"]])#media
imp = SimpleImputer(missing_values=np.nan, strategy='median')
dados_hist["media_final_median"] = imp.fit_transform(dados_hist[["media_final_median"]])#mediana
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
dados_hist["media_final_frequency"] = imp.fit_transform(dados_hist[["media_final_frequency"]])#valor mais frequente

Criando nosso dataset de trabalho, o dados_aluno

In [ ]:
col=["alunoid"]
dados_aluno=pd.DataFrame(columns=col)
dados_aluno=dados_matr#ele espelha o dataset matricula

dados_aluno["reprovacoes"]=dados_hist.loc[(dados_hist.situacao=="Reprovado")].groupby(by="alunoid")["situacao"].count()#atributo criado, conta o numero de reprovações por nota
dados_aluno["reprovacoes_por_falta"]=dados_hist.loc[(dados_hist.situacao=="Reprov. por Falta")|(dados_hist.situacao=="Reprovado por falta")].groupby(by="alunoid")["situacao"].count()#atributo criado, conta o numero de reprovações por falta
dados_aluno["percentual_carga_horaria_frequentada_mean"]=dados_hist.groupby(by="alunoid").percentual_carga_horaria_frequentada.mean()#atributo criado, calcula a media de percentual de carga horaria frequentada pelo aluno
dados_aluno["carga_horaria_total"]=dados_hist.groupby(by="alunoid").carga_horaria.sum()#atributo criado, calcula a soma das cargas horarias frequentadas
dados_aluno.loc[dados_aluno.carga_horaria_total.isnull(),"carga_horaria_total"]=0#iguala os valores nulos a zero
dados_aluno.loc[dados_aluno.percentual_carga_horaria_frequentada_mean.isnull(),"percentual_carga_horaria_frequentada_mean"]=0#iguala os valores nulos a zero
dados_aluno.loc[dados_aluno.reprovacoes.isnull(),"reprovacoes"]=0#iguala os valores nulos a zero
dados_aluno.loc[dados_aluno.reprovacoes_por_falta.isnull(),"reprovacoes_por_falta"]=0#iguala os valores nulos a zero

Padronização dos valores dos nossos atributos para minusculas

In [ ]:
dados_aluno = dados_aluno.astype(str).apply(lambda x: x.str.lower())

Conversao do tipo de nossas colunas para numerico

In [ ]:
#esse comando tenta passar todos as colunas para numerico. Se ele encontra alguma coluna com string, não vai conseguir e então ele ignora essa coluna
dados_aluno=dados_aluno.apply(pd.to_numeric, errors='ignore')

Dropando colunas com dados irrelevantes ou com dificuldade muito elevada de imputação de valores

In [ ]:
dados_aluno=dados_aluno.drop(columns=["dataconclusao","forma_acesso_seletivo","razao_ausencia_educacional","exclusivo_rede_publica"])
dados_aluno=dados_aluno.drop(dados_aluno.situacao=="matriculado")|(dados_aluno.situacao=="intercâmbio")|
           (dados_aluno.situacao=="matrícula vínculo institucional")|(dados_aluno.situacao=="transferido externo")|
           (dados_aluno.situacao=="transferido interno")]=0

SyntaxError: ignored

Valores para nossa classe objetivo

In [ ]:
dados_aluno.situacao.loc[(dados_aluno.situacao=="aguardando ENADE")|(dados_aluno.situacao=="aguardando enade")|(dados_aluno.situacao=="aguardando colação de grau")|
           (dados_aluno.situacao=="estagiario (concludente)")|(dados_aluno.situacao=="concludente")|(dados_aluno.situacao=="formado")|
           (dados_aluno.situacao=="concluído")|(dados_aluno.situacao=="matriculado")|(dados_aluno.situacao=="intercâmbio")|
           (dados_aluno.situacao=="matrícula vínculo institucional")|(dados_aluno.situacao=="transferido externo")|
           (dados_aluno.situacao=="transferido interno")]=0
dados_aluno.situacao.loc[(dados_aluno.situacao=="falecido")|(dados_aluno.situacao=="cancelamento por desligamento")|(dados_aluno.situacao=="afastado")|
           (dados_aluno.situacao=="cancelamento por duplicidade")|(dados_aluno.situacao=="jubilado")|(dados_aluno.situacao=="trancada")|
           (dados_aluno.situacao=="cancelamento compulsório")|(dados_aluno.situacao=="trancado voluntariamente")|(dados_aluno.situacao=="cancelado")|
           (dados_aluno.situacao=="evasão")]=1

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Transformação de nossa classe alvo para boleano

In [ ]:
# Por quê?
dados_aluno["situacao"]=dados_aluno["situacao"].astype(bool)

In [ ]:
# em nossa coluna raca, tinhamos o mesmo valor escrito de formas diferentes
dados_aluno.loc[dados_aluno.raca == "indígena","raca"]="indigena"

In [ ]:
dados_aluno=dados_aluno.drop(columns="alunoid")

Reindexação de nosso dataset, para que a classe alvo seja a ultima a aparecer

In [ ]:
dados_aluno = dados_aluno.reindex(columns=['campus','curso','anoingresso','periodoingresso',
 'rendabruta','ira','modalidade','genero','raca','idade','idioma','ficou_tempo_sem_estudar',
 'quantidade_computadores','companhia_domiciliar','mae_nivel_escolaridade','pai_nivel_escolaridade',
 'quantidade_notebooks','estado_civil','qtd_filhos','tipo_area_residencial','trabalha','reprovacoes',
 'reprovacoes_por_falta','percentual_carga_horaria_frequentada_mean','carga_horaria_total','situacao'])

In [ ]:
dados_aluno.select_dtypes(include="object")

,campus,curso,modalidade,genero,raca,idioma,ficou_tempo_sem_estudar,companhia_domiciliar,mae_nivel_escolaridade,pai_nivel_escolaridade,estado_civil,tipo_area_residencial,trabalha
0,mtc,licenciatura em biologia,licenciatura,m,preta,false,false,pai,alfabetizado,alfabetizado,solteiro(a),rural,serviço público
1,mtc,técnico em eletromecânica tc: nárea: controle ...,concomitante,m,parda,true,false,tios,ensino médio completo,ensino médio completo,solteiro(a),urbano,nunca trabalhou
2,mtc,técnico em eletrotécnica tc: t nárea: controle...,concomitante,m,parda,false,false,mãe,ensino médio completo,ensino médio completo,solteiro(a),rural,nunca trabalhou
3,mar,curso técnico em informática ead,concomitante,m,parda,true,false,tios,ensino médio completo,ensino médio completo,solteiro(a),urbano,nunca trabalhou
4,tmn,timon - técnico em eletromecânica integrado,integrado,m,parda,true,false,pai,ensino superior incompleto,ensino médio incompleto,solteiro(a),urbano,nunca trabalhou
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34516,mar,técnico em agropecuária - ead,subsequente,m,parda,false,false,cônjuge,não estudou,não estudou,casado(a),rural,trabalhador rural/agricultor
34517,pdu,técnico em administração,subsequente,m,branca,false,false,parentes,alfabetizado,alfabetizado,solteiro(a),urbano,não está trabalhando
34518,cch,curso técnico em manutenção de máquinas indust...,integrado,f,parda,false,false,parentes,alfabetizado,ensino fundamental completo,solteiro(a),urbano,beneficiário ou pensionista do inss
34519,cod,bacharelado em agronomia,bacharelado,m,preta,false,false,mãe,ensino médio incompleto,ensino médio incompleto,união estável,urbano,trabalho informal/bico


Separação entre previsores e situacao

In [ ]:
#profile = ProfileReport(dados_aluno, title='Profiling Dados Aluno',html={'style':{'full_width':True}})
#profile.to_notebook_iframe()

In [ ]:
previsores=dados_aluno.iloc[:,0:25].values #as colunas que nos ajudarao a prever o comportamento do aluno 
situacao=dados_aluno.iloc[:,25].values #coluna alvo

In [ ]:
import pickle
from sklearn.preprocessing import OneHotEncoder

# Load data and save indices of columns
df = dados_aluno
features = df.drop('situacao', 1).columns
pickle.dump(features, open('features.pickle', 'wb'))

# Fit and save an OneHotEncoder
columns_to_fit = ['campus', 'curso', 'anoingresso', 'periodoingresso', 'rendabruta','ira', 'modalidade', 'genero', 'idade', 'ficou_tempo_sem_estudar','companhia_domiciliar', 'estado_civil', 'trabalha']
enc = OrdinalEncoder().fit(df.loc[:, columns_to_fit])
pickle.dump(enc, open('encoder.pickle', 'wb'))

# Transform variables, merge with existing df and keep column names
column_names = columns_to_fit
encoded_variables = pd.DataFrame(enc.transform(df.loc[:, columns_to_fit]), columns=column_names)
df = df.drop(columns_to_fit, 1)
df = pd.concat([df, encoded_variables], axis=1)


    
# Fit and save model
# X, y = df.drop('situacao', 1), df.loc[:, 'situacao']
# clf = GaussianNB().fit(X, y)
# pickle.dump(clf, open('model.pickle', 'wb'))

Tratamento de valores categóricos, usando o labelencoder

In [ ]:
# label_encoder_campus = LabelEncoder()
# label_encoder_curso = LabelEncoder()
# label_encoder_modalidade = LabelEncoder()
# label_encoder_genero = LabelEncoder()
# label_encoder_raca = LabelEncoder()
# label_encoder_idioma = LabelEncoder()
# label_encoder_ftse = LabelEncoder()
# label_encoder_cia_domiciliar = LabelEncoder()
# label_encoder_nivel_pai = LabelEncoder()
# label_encoder_nivel_mae = LabelEncoder()
# label_encoder_estado_civil = LabelEncoder()
# label_encoder_tipo_residencia = LabelEncoder()
# label_encoder_trabalha = LabelEncoder()

In [ ]:
# previsores[:,0] = label_encoder_campus.fit_transform(previsores[:,0])
# previsores[:,1] = label_encoder_curso.fit_transform(previsores[:,1])
# previsores[:,6] = label_encoder_modalidade.fit_transform(previsores[:,6])
# previsores[:,7] = label_encoder_genero.fit_transform(previsores[:,7])
# previsores[:,8] = label_encoder_raca.fit_transform(previsores[:,8])
# previsores[:,10] = label_encoder_idioma.fit_transform(previsores[:,10])
# previsores[:,11] = label_encoder_ftse.fit_transform(previsores[:,11])
# previsores[:,13] = label_encoder_cia_domiciliar.fit_transform(previsores[:,13])
# previsores[:,14] = label_encoder_nivel_pai.fit_transform(previsores[:,14])
# previsores[:,15] = label_encoder_nivel_mae.fit_transform(previsores[:,15])
# previsores[:,17] = label_encoder_estado_civil.fit_transform(previsores[:,17])
# previsores[:,19] = label_encoder_tipo_residencia.fit_transform(previsores[:,19])
# previsores[:,20] = label_encoder_trabalha.fit_transform(previsores[:,20])

Standardização dos valores previsores

In [ ]:
# A=set(dados_aluno.columns)
# B=set(columns_to_fit.pop())
columns_to_scale=["rendabruta",	"ira", "idade", "quantidade_computadores", "quantidade_notebooks", "qtd_filhos"]

# Nao deveria ser aplicado apenas para as colunas que não sofreram codificação OneHot? Ou seja, apenas as numericas devem ser normalizadas
# Mais precisamente, só entendo que devem ser normalizados os campos: rendabruta,	ira, idade, quantidade_computadores, quantidade_notebooks, qtd_filhos
scaler = StandardScaler()  

# que tal chamar s_previsores, pra indicar que foram scaled?
s_valores = scaler.fit_transform(df.loc[:, columns_to_scale])#valores normalizados
s_df=pd.DataFrame(data=s_valores[0:,0:],
                index=[i for i in range(s_valores.shape[0])],
                columns=columns_to_scale)
df = df.drop(columns_to_scale, 1)
df = pd.concat([df, s_df], axis=1)

df = df.reindex(columns=['campus','curso','anoingresso','periodoingresso',
 'rendabruta','ira','modalidade','genero','raca','idade','idioma','ficou_tempo_sem_estudar',
 'quantidade_computadores','companhia_domiciliar','mae_nivel_escolaridade','pai_nivel_escolaridade',
 'quantidade_notebooks','estado_civil','qtd_filhos','tipo_area_residencial','trabalha','reprovacoes',
 'reprovacoes_por_falta','percentual_carga_horaria_frequentada_mean','carga_horaria_total','situacao'])

pickle.dump(scaler, open('scaler.pkl', 'wb'))

Divisão entre teste/treinamento

In [ ]:
# por quê usar aqui series de valores e não o dataframe?
previsores_treinamento, previsores_teste, situacao_treinamento, situacao_teste = train_test_split(df.drop('situacao', 1), df.loc[:, 'situacao'], test_size = 0.15, random_state = 0)

# Naive Bayes

Usando todos os features que temos

In [ ]:
naive = GaussianNB()
naive.fit(previsores_treinamento, situacao_treinamento)
previsoes = naive.predict(previsores_teste)
previsores_treinamento

,campus,curso,anoingresso,periodoingresso,rendabruta,ira,modalidade,genero,raca,idade,idioma,ficou_tempo_sem_estudar,quantidade_computadores,companhia_domiciliar,mae_nivel_escolaridade,pai_nivel_escolaridade,quantidade_notebooks,estado_civil,qtd_filhos,tipo_area_residencial,trabalha,reprovacoes,reprovacoes_por_falta,percentual_carga_horaria_frequentada_mean,carga_horaria_total
7975,12,114,2017,1,-0.205584,0.689285,3,2,2,-0.580360,1,1,-0.200040,1,4,4,-0.592698,1,-0.319728,2,2,0.0,0.0,0.0,0.0
10524,20,169,2015,2,-0.449448,0.650134,1,2,2,1.858004,1,1,1.179972,6,1,7,1.257762,1,6.372800,1,1,0.0,0.0,0.0,0.0
2140,12,114,2016,1,-0.099247,0.966139,3,2,2,-0.293494,1,1,-0.200040,3,2,4,-0.592698,1,-0.319728,2,2,0.0,0.0,0.0,0.0
8130,20,196,2014,2,0.077980,0.546664,1,1,5,0.567106,1,1,-0.200040,5,5,7,-0.592698,1,-0.319728,2,1,0.0,0.0,0.0,0.0
31615,6,198,2017,1,0.042534,0.432007,4,2,2,-0.150060,1,1,-0.200040,4,6,4,-0.592698,1,-0.319728,1,2,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20757,4,50,2019,2,-0.241029,0.373281,1,1,2,-0.293494,1,1,-0.200040,4,2,2,1.257762,1,-0.319728,2,2,0.0,0.0,0.0,0.0
32103,17,37,2017,1,-0.141782,0.571832,3,1,3,-0.293494,1,1,-0.200040,4,2,2,1.257762,1,-0.319728,2,2,0.0,0.0,0.0,0.0
30403,18,232,2017,1,-0.170138,0.557850,3,2,2,-0.436927,1,1,-0.200040,4,6,1,-0.592698,1,-0.319728,2,2,0.0,0.0,0.0,0.0
21243,21,97,2015,1,-0.329643,0.717250,3,1,2,-0.436927,1,1,-0.200040,5,5,4,1.257762,1,-0.319728,2,2,0.0,0.0,0.0,0.0


In [ ]:
accuracy_score(situacao_teste, previsoes)

0.654759606101564

In [ ]:
print(classification_report(situacao_teste, previsoes))

NameError: ignored

In [ ]:
confusion_matrix(situacao_teste, previsoes)

NameError: ignored

## Naive Bayes com seleção de atributos

Usando o algoritmo f_class

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif, mutual_info_classif
f_classif = SelectKBest(score_func=f_classif, k=10)
fit = f_classif.fit(df.drop('situacao', 1),situacao)
previsores_fclass = fit.transform(df.drop('situacao', 1))

cols = fit.get_support(indices=True)


In [ ]:
previsores_fclass_treinamento, previsores_fclass_teste, situacao_treinamento, situacao_teste = train_test_split(previsores_fclass, situacao, test_size = 0.15, random_state = 0)

In [ ]:
naive_fclass = GaussianNB()
naive_fclass.fit(previsores_fclass_treinamento, situacao_treinamento)
previsoes_fclass = naive_fclass.predict(previsores_fclass_teste)
previsores_fclass_treinamento.shape

(29342, 10)

In [ ]:
accuracy_score(situacao_teste, previsoes_fclass)

0.8520949990345627

In [ ]:
confusion_matrix(situacao_teste, previsoes_fclass)

Usando o mutual_info_class

In [ ]:
#a tayna usou um algoritmo parecido, o information gain. Segundo pesquisas o information gain é mais usado em arvores de decisao

In [ ]:
from sklearn.feature_selection import f_classif, mutual_info_classif
mutual_info_classif = SelectKBest(score_func=mutual_info_classif, k=10)
fit = mutual_info_classif.fit(df.drop('situacao', 1),situacao)
previsores_mutual_info_class = fit.transform(df.drop('situacao', 1))

cols = fit.get_support(indices=True)
pickle.dump(df.iloc[:,cols], open('features_esc.pkl', 'wb'))

In [ ]:
previsores_mutualinfoclass_treinamento, previsores_mutualinfoclass_teste, situacao_treinamento, situacao_teste = train_test_split(previsores_mutual_info_class, situacao, test_size = 0.15, random_state = 0)

In [ ]:
naive_mutualinfoclass = GaussianNB()
naive_mutualinfoclass.fit(previsores_mutualinfoclass_treinamento, situacao_treinamento)
pickle.dump(naive_mutualinfoclass, open('modelo.pickle', 'wb'))
print(previsores_mutualinfoclass_teste)
previsoes_mutualinfoclass = naive_mutualinfoclass.predict(previsores_mutualinfoclass_teste)
previsoes_mutualinfoclass

In [ ]:
accuracy_score(situacao_teste, previsoes_mutualinfoclass)

In [ ]:
confusion_matrix(situacao_teste, previsoes_mutualinfoclass)

Salvamento do modelo treinado usando o pickle

In [ ]:
previsores_mutualinfoclass_teste
situacao_teste

In [ ]:
# pickle.dump(previsores_mutualinfoclass_teste, open("previsores_teste.sav","wb"))
# pickle.dump(situacao_teste, open("situacao_teste.sav","wb"))

In [ ]:
# import pickle

# previsores_mutualinfoclass=np.concatenate((previsores_mutualinfoclass_treinamento,previsores_mutualinfoclass_teste),axis=0)
# situacao_mutualinfoclass=np.concatenate((situacao_treinamento,situacao_teste),axis=0)
# naive_finalizado=GaussianNB()
# naive_finalizado.fit(previsores_mutualinfoclass,situacao_mutualinfoclass)
# pickle.dump(naive_mutualinfoclass, open("classificador_naive_mutualinfoclass.sav","wb"))

Salvamento do modelo treinado usando o joblib

In [ ]:
from sklearn.externals import joblib
joblib.dump(naive_mutualinfoclass, "classificador_naive_mutualinfoclass_jl.joblib")

# Árvores de Decisão

Usando todas as features

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
arvore = DecisionTreeClassifier(criterion='entropy', random_state = 0)
arvore.fit(previsores_treinamento, situacao_treinamento)

In [ ]:
previsoes_tree = arvore.predict(previsores_teste)
previsoes_tree

In [ ]:
accuracy_score(situacao_teste, previsoes_tree)

In [ ]:
print(classification_report(situacao_teste, previsoes_tree))

In [ ]:
confusion_matrix(situacao_teste, previsoes_tree)

## Arvores de decisão com seleção de features

Usando o algoritmo f_class

In [ ]:
f_classif_tree = SelectKBest(score_func=f_classif, k=10)
fit = f_classif_tree.fit(previsores,situacao)
previsores_fclass_tree = fit.transform(previsores)

cols = fit.get_support(indices=True)
dados_aluno.iloc[:,cols]

In [ ]:
previsores_fclass_tree_treinamento, previsores_fclass_tree_teste, situacao_treinamento, situacao_teste = train_test_split(previsores_fclass_tree, situacao, test_size = 0.15, random_state = 0)

In [ ]:
tree_fclass = DecisionTreeClassifier(criterion='entropy', random_state = 0)
tree_fclass.fit(previsores_fclass_tree_treinamento, situacao_treinamento)
previsoes_fclass_tree = tree_fclass.predict(previsores_fclass_tree_teste)
previsoes_fclass_tree

In [ ]:
accuracy_score(situacao_teste, previsoes_fclass_tree)

In [ ]:
confusion_matrix(situacao_teste, previsoes_fclass_tree)

Usando o mutual_info_class

In [ ]:
from sklearn.feature_selection import  mutual_info_classif
mutual_info_classif_tree = SelectKBest(score_func=mutual_info_classif, k=10)
fit = mutual_info_classif_tree.fit(previsores,situacao)
previsores_mutual_info_class_tree = fit.transform(previsores)

cols = fit.get_support(indices=True)
dados_aluno.iloc[:,cols]

In [ ]:
previsores_mutualinfoclass_tree_treinamento, previsores_mutualinfoclass_tree_teste, situacao_treinamento, situacao_teste = train_test_split(previsores_mutual_info_class_tree, situacao, test_size = 0.15, random_state = 0)

In [ ]:
tree_mutualinfoclass = DecisionTreeClassifier(criterion='entropy', random_state = 0)
tree_mutualinfoclass.fit(previsores_mutualinfoclass_tree_treinamento, situacao_treinamento)
previsoes_mutualinfoclass_tree = tree_mutualinfoclass.predict(previsores_mutualinfoclass_tree_teste)
previsoes_mutualinfoclass_tree

In [ ]:
accuracy_score(situacao_teste, previsoes_mutualinfoclass_tree)

In [ ]:
confusion_matrix(situacao_teste, previsoes_mutualinfoclass_tree)

Usando o chi2

In [ ]:
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
previsores_minmax = minmax.fit_transform(previsores)

In [ ]:
from sklearn.feature_selection import  chi2
chi2_tree = SelectKBest(score_func=chi2, k=10)
fit = chi2_tree.fit(previsores_minmax,situacao)
previsores_chi2_tree = fit.transform(previsores_minmax)

cols = fit.get_support(indices=True)
dados_aluno.iloc[:,cols]

In [ ]:
previsores_chi2_tree_treinamento, previsores_chi2_tree_teste, situacao_treinamento, situacao_teste = train_test_split(previsores_chi2_tree, situacao, test_size = 0.15, random_state = 0)

In [ ]:
tree_chi2 = DecisionTreeClassifier(criterion='entropy', random_state = 0)
tree_chi2.fit(previsores_chi2_tree_treinamento, situacao_treinamento)
previsoes_chi2_tree = tree_chi2.predict(previsores_chi2_tree_teste)
previsoes_chi2_tree

In [ ]:
accuracy_score(situacao_teste, previsoes_chi2_tree)

In [ ]:
confusion_matrix(situacao_teste, previsoes_chi2_tree)

#Random Forest

Usando todas as features

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
random_forest = RandomForestClassifier(n_estimators=40, criterion='entropy', random_state = 0)
random_forest.fit(previsores_treinamento, situacao_treinamento)

In [ ]:
previsoes_random = random_forest.predict(previsores_teste)
previsoes_random

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
accuracy_score(situacao_teste, previsoes_random)

In [ ]:
print(classification_report(situacao_teste, previsoes_random))

In [ ]:
confusion_matrix(situacao_teste, previsoes_random)

## Random Forests usando seleção de features

Usando f_class

In [ ]:
f_classif_random = SelectKBest(score_func=f_classif, k=10)
fit = f_classif_random.fit(previsores,situacao)
previsores_fclass_random = fit.transform(previsores)

In [ ]:
previsores_fclass_random_treinamento, previsores_fclass_random_teste, situacao_treinamento, situacao_teste = train_test_split(previsores_fclass_random, situacao, test_size = 0.15, random_state = 0)

In [ ]:
random_fclass = RandomForestClassifier(n_estimators=40, criterion='entropy', random_state = 0)
random_fclass.fit(previsores_fclass_random_treinamento, situacao_treinamento)
previsoes_fclass_random = random_fclass.predict(previsores_fclass_random_teste)
previsoes_fclass_random

In [ ]:
accuracy_score(situacao_teste, previsoes_fclass_random)

In [ ]:
print(classification_report(situacao_teste, previsoes_fclass_random))

In [ ]:
confusion_matrix(situacao_teste, previsoes_fclass_random)

Usando o mutual_info_class

In [ ]:
from sklearn.feature_selection import  mutual_info_classif
mutual_info_classif_random = SelectKBest(score_func=mutual_info_classif, k=10)
fit = mutual_info_classif_random.fit(previsores,situacao)
previsores_mutual_info_class_random = fit.transform(previsores)

In [ ]:
previsores_mutualinfoclass_random_treinamento, previsores_mutualinfoclass_random_teste, situacao_treinamento, situacao_teste = train_test_split(previsores_mutual_info_class_random, situacao, test_size = 0.15, random_state = 0)

In [ ]:
random_mutualinfoclass = RandomForestClassifier(n_estimators=40, criterion='entropy', random_state = 0)
random_mutualinfoclass.fit(previsores_mutualinfoclass_random_treinamento, situacao_treinamento)
previsoes_mutualinfoclass_random = random_mutualinfoclass.predict(previsores_mutualinfoclass_random_teste)
previsoes_mutualinfoclass_random

In [ ]:
accuracy_score(situacao_teste, previsoes_mutualinfoclass_random)

In [ ]:
confusion_matrix(situacao_teste, previsoes_mutualinfoclass_random)

Usando o chi2

In [ ]:
from sklearn.feature_selection import  chi2
chi2_random = SelectKBest(score_func=chi2, k=10)
fit = chi2_random.fit(previsores_minmax,situacao)
previsores_chi2_random = fit.transform(previsores_minmax)

In [ ]:
previsores_chi2_random_treinamento, previsores_chi2_random_teste, situacao_treinamento, situacao_teste = train_test_split(previsores_chi2_random, situacao, test_size = 0.15, random_state = 0)

In [ ]:
random_chi2 = RandomForestClassifier(n_estimators=40, criterion='entropy', random_state = 0)
random_chi2.fit(previsores_chi2_random_treinamento, situacao_treinamento)
previsoes_chi2_random = random_chi2.predict(previsores_chi2_random_teste)
previsoes_chi2_tree

In [ ]:
accuracy_score(situacao_teste, previsoes_chi2_random)

In [ ]:
confusion_matrix(situacao_teste, previsoes_chi2_random)

#SVM

Usando todas as features

In [ ]:
from sklearn.svm import SVC

In [ ]:
svm = SVC(kernel='rbf', random_state=1, C = 2.0) # 2 -> 4
svm.fit(previsores_treinamento, situacao_treinamento)

In [ ]:
previsoes_svm = svm.predict(previsores_teste)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
accuracy_score(situacao_teste, previsoes_svm)

In [ ]:
print(classification_report(situacao_teste, previsoes_svm))

In [ ]:
confusion_matrix(situacao_teste, previsoes_svm)

##SVM usando seleção de features

Usando o f_class

In [ ]:
f_classif_svm = SelectKBest(score_func=f_classif, k=10)
fit = f_classif_svm.fit(previsores,situacao)
previsores_fclass_svm = fit.transform(previsores)

In [ ]:
previsores_fclass_svm_treinamento, previsores_fclass_svm_teste, situacao_treinamento, situacao_teste = train_test_split(previsores_fclass_svm, situacao, test_size = 0.15, random_state = 0)

In [ ]:
svm_fclass = SVC(kernel='rbf', random_state=1, C = 2.0)
svm_fclass.fit(previsores_fclass_svm_treinamento, situacao_treinamento)
previsoes_fclass_svm = svm_fclass.predict(previsores_fclass_svm_teste)
previsoes_fclass_svm

In [ ]:
accuracy_score(situacao_teste, previsoes_fclass_svm)

In [ ]:
print(classification_report(situacao_teste, previsoes_fclass_svm))

In [ ]:
confusion_matrix(situacao_teste, previsoes_fclass_svm)

Usando o mutual_info_class

In [ ]:
from sklearn.feature_selection import  mutual_info_classif
mutual_info_classif_svm = SelectKBest(score_func=mutual_info_classif, k=10)
fit = mutual_info_classif_svm.fit(previsores,situacao)
previsores_mutual_info_class_svm = fit.transform(previsores)

In [ ]:
previsores_mutualinfoclass_svm_treinamento, previsores_mutualinfoclass_svm_teste, situacao_treinamento, situacao_teste = train_test_split(previsores_mutual_info_class_svm, situacao, test_size = 0.15, random_state = 0)

In [ ]:
svm_mutualinfoclass = SVC(kernel='rbf', random_state=1, C = 2.0)
svm_mutualinfoclass.fit(previsores_mutualinfoclass_svm_treinamento, situacao_treinamento)
previsoes_mutualinfoclass_svm = svm_mutualinfoclass.predict(previsores_mutualinfoclass_svm_teste)
previsoes_mutualinfoclass_svm

In [ ]:
accuracy_score(situacao_teste, previsoes_mutualinfoclass_svm)

In [ ]:
confusion_matrix(situacao_teste, previsoes_mutualinfoclass_svm)

Usando o chi2

In [ ]:
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
previsores_minmax = minmax.fit_transform(previsores)

In [ ]:
from sklearn.feature_selection import  chi2
chi2_svm = SelectKBest(score_func=chi2, k=10)
fit = chi2_svm.fit(previsores_minmax,situacao)
previsores_chi2_svm = fit.transform(previsores_minmax)

In [ ]:
previsores_chi2_svm_treinamento, previsores_chi2_svm_teste, situacao_treinamento, situacao_teste = train_test_split(previsores_chi2_svm, situacao, test_size = 0.15, random_state = 0)

In [ ]:
svm_chi2 = RandomForestClassifier(n_estimators=40, criterion='entropy', random_state = 0)
svm_chi2.fit(previsores_chi2_svm_treinamento, situacao_treinamento)
previsoes_chi2_svm = svm_chi2.predict(previsores_chi2_svm_teste)
previsoes_chi2_svm

In [ ]:
accuracy_score(situacao_teste, previsoes_chi2_svm)

In [ ]:
confusion_matrix(situacao_teste, previsoes_chi2_svm)

# Análise de Dados

# Aprendizagem 

# Distribuição do Modelo